<a href="https://colab.research.google.com/github/shuklaashi2303/FinalProjectStock/blob/main/Stock_Return_StandardScalar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install yfinance

In [2]:
# Import Depandancies for Data Extraction
import pandas as pd
import yfinance as yf
import datetime
import plotly.express as px
import plotly.graph_objects as go

In [3]:
url = 'https://raw.githubusercontent.com/ritwikthakar/test/main/constituents.csv'
spx_data_df = pd.read_csv(url)
spx_data = spx_data_df.dropna()

In [4]:
# Defining Parametres for Main Data Frame 
symbol_df = spx_data['Symbol']
start = datetime.datetime.today()-datetime.timedelta(3650)
end = datetime.datetime.today()
cl_price = pd.DataFrame()

In [5]:
# Data Extraction with YFinance API
for ticker in symbol_df:
  cl_price[ticker] = yf.download(ticker, start, end)['Adj Close']

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [6]:
# Creating Main Data Frame for Machine Learning
main_df_raw = cl_price.dropna(axis = 1)
main_df = main_df_raw.pct_change().dropna()
main_df

,MMM,AOS,ABT,ABMD,ACN,ATVI,ADM,ADBE,AAP,AMD,AES,AFL,A,APD,AKAM,ALK,ALB,ARE,ALGN,LNT,ALL,GOOGL,GOOG,MO,AMZN,AEE,AAL,AEP,AXP,AIG,AMT,AWK,AMP,ABC,AME,AMGN,APH,ADI,ANSS,ANTM,...,URI,UNH,UHS,UNM,VLO,VTR,VRSN,VRSK,VZ,VRTX,VFC,VIAC,VTRS,V,VNO,VMC,WRB,GWW,WAB,WBA,WMT,WM,WAT,WEC,WFC,WELL,WST,WDC,WU,WY,WHR,WMB,WLTW,WYNN,XEL,XLNX,YUM,ZBRA,ZBH,ZION
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2011-08-02,-0.017287,-0.028359,-0.005350,-0.043533,-0.016113,-0.013502,-0.061680,-0.023130,-0.017712,-0.015235,-0.036946,-0.024726,-0.045354,-0.037276,-0.031132,-0.049942,-0.050353,-0.046437,-0.046050,-0.012094,-0.024700,-0.023683,-0.023683,0.001147,-0.043467,-0.022092,-0.073826,-0.019287,-0.029988,-0.029381,-0.038095,-0.014617,-0.031885,0.000000,-0.042608,-0.008555,-0.038652,-0.032277,-0.023658,-0.022922,...,-0.072833,-0.021657,-0.010204,-0.026012,-0.055333,-0.014937,-0.021533,-0.041159,-0.010594,-0.037102,-0.033696,-0.036657,-0.058691,-0.024744,-0.051023,-0.062612,-0.016580,-0.039850,-0.041756,-0.022251,-0.017864,-0.026048,-0.051454,-0.015008,-0.026853,-0.003312,-0.020484,-0.051230,-0.031447,-0.033829,-0.039947,-0.039544,-0.011164,-0.047857,-0.020799,-0.027638,-0.031902,-0.018519,-0.021353,-0.044913
2011-08-03,0.010672,0.006431,0.001793,0.005277,0.027409,0.011121,0.036364,0.012579,0.020849,0.007032,-0.003410,0.006506,0.006677,0.013493,0.025619,0.026283,0.002374,-0.013527,-0.026503,0.000511,-0.002533,0.014804,0.014804,0.006876,-0.008219,0.007765,0.068841,0.000000,0.007008,0.015856,0.062772,0.023517,-0.003939,0.011397,0.004918,0.000187,0.010750,0.015767,0.016901,0.012356,...,-0.012048,0.004257,-0.000219,0.033065,-0.012733,-0.015746,0.014005,0.062957,0.013525,-0.023445,0.016770,0.015982,-0.021103,0.046913,-0.006621,0.049618,0.006942,0.009545,0.005447,-0.002355,-0.007740,0.006523,0.012340,-0.005962,0.008869,0.005607,0.006179,0.011288,0.022186,0.005835,0.027484,-0.009882,0.004516,0.033508,-0.003823,0.014535,0.001373,-0.003102,0.004364,0.002879
2011-08-04,-0.045834,-0.043500,-0.030225,-0.209318,-0.043960,-0.021996,-0.053644,-0.044574,-0.029623,-0.092179,-0.080411,-0.057958,-0.058418,-0.049780,-0.031329,-0.053270,-0.093478,-0.051848,-0.059310,-0.053786,-0.045702,-0.039340,-0.039340,-0.033005,-0.040389,-0.054641,-0.045763,-0.029364,-0.041343,-0.063498,-0.049562,-0.024390,-0.052590,-0.049790,-0.064351,-0.033008,-0.073389,-0.055821,0.022026,-0.038159,...,-0.149268,-0.044722,-0.071523,-0.052113,-0.097163,-0.042663,-0.037488,-0.013161,-0.029191,-0.077453,-0.047735,-0.093259,-0.065654,-0.028464,-0.044932,0.016364,-0.020026,-0.038629,-0.078872,-0.028579,-0.023011,-0.025599,-0.048153,-0.028991,-0.057143,-0.023131,-0.017242,-0.064857,-0.054526,-0.045886,-0.034220,-0.100465,-0.054196,-0.070379,-0.027292,-0.043298,-0.012928,-0.083744,-0.018074,-0.071770
2011-08-05,0.006324,-0.021582,0.029936,-0.042324,0.004036,-0.011246,0.021034,-0.020268,0.003792,0.009231,0.006512,-0.012778,-0.008670,0.002071,-0.012675,-0.006853,-0.019683,-0.029889,-0.012920,-0.007813,-0.000760,0.002632,0.002632,0.015693,0.006055,0.023342,-0.019538,0.005829,0.007899,-0.049242,-0.020192,-0.006884,-0.015442,0.015168,-0.031381,0.010861,-0.004132,-0.004110,-0.030564,-0.004979,...,0.012041,0.010872,-0.010870,-0.013420,-0.013810,-0.017330,-0.011616,0.005759,0.003723,-0.026929,0.020619,-0.018587,0.009963,-0.018590,-0.028035,0.006261,-0.011391,0.013687,-0.007265,-0.000540,0.014970,0.009977,-0.010904,0.016129,-0.020591,-0.018997,0.007210,-0.014839,-0.001120,-0.012161,0.026768,0.000370,-0.018748,-0.008223,0.018413,-0.003993,0.006350,-0.019525,0.018761,-0.013917
2011-08-08,-0.050272,-0.070378,-0.043201,-0.078856,-0.070792,-0.046369,-0.082053,-0.066745,-0.054779,-0.094512,-0.112754,-0.096836,-0.110139,-0.060791,-0.066401,-0.051934,-0.085643,-0.110748,-0.098953,-0.068694,-0.065424,-0.057025,-0.057025,-0.052144,-0.044401,-0.059740,-0.099637,-0.052264,-0.088329,-0.100398,-0.072629,-0.067859,-0.123781,-0.059223,-0.089093,-0.042978,-0.050945,-0.041905,-0.069725,-0.079742,...

In [7]:
stock_df = main_df['MMM']
stock_data = main_df['MMM'].reset_index()

In [8]:
#Plot Current Stock Returns
fig = px.line(stock_data, x="Date", y="MMM")


fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1,
                     label="1m",
                     step="month",
                     stepmode="backward"),
                dict(count=6,
                     label="6m",
                     step="month",
                     stepmode="backward"),
                dict(count=1,
                     label="YTD",
                     step="year",
                     stepmode="todate"),
                dict(count=1,
                     label="1y",
                     step="year",
                     stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(
            visible=True
        ),
        type="date"
    )
)

fig.show()

In [9]:
# Import Dependancies for Data Preprocessing for Machine Learning
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [10]:
# Transforming Data in Numpy Array
scaler = StandardScaler()
stock_df1 = scaler.fit_transform(np.array(stock_df).reshape(-1,1))

In [11]:
##splitting dataset into train and test split
training_size=int(len(stock_df1)*0.65)
test_size=len(stock_df1)-training_size
train_data,test_data=stock_df1[0:training_size,:],stock_df1[training_size:len(stock_df1),:1]
training_size,test_size

(1633, 880)

In [12]:
# Function for Creating Data Set Arrays 
def create_dataset(dataset, time_step=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-time_step-1):
        a = dataset[i:(i+time_step), 0]  
        dataX.append(a)
        dataY.append(dataset[i + time_step, 0])
    return np.array(dataX), np.array(dataY)

In [13]:
# Creation of Testing & Training Data 
time_step = 100
X_train, y_train = create_dataset(train_data, time_step)
X_test, y_test = create_dataset(test_data, time_step)

In [14]:
# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [15]:
# Import Dependancies for Deep Learning
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
import math
from sklearn.metrics import mean_squared_error
from sklearn import metrics

In [16]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features= len(X_train[0])
hidden_layers1= 50
hidden_layers2= 45


nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_layers1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_layers2, input_dim=number_input_features, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 50)                5050      
_________________________________________________________________
dense_1 (Dense)              (None, 45)                2295      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 46        
Total params: 7,391
Trainable params: 7,391
Non-trainable params: 0
_________________________________________________________________


In [17]:
# Compile the model
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

nn.compile(loss="binary_crossentropy", optimizer="adam", metrics= ["accuracy"])

In [18]:
# Create a callback which saves the weights for every 5 epochs
cp_callback= ModelCheckpoint(filepath=checkpoint_path,save_weights_only=True, save_freq=5)

In [19]:
# Train the model
fit_model= nn.fit(X_train_scaled, y_train, epochs=100, verbose=1, callbacks=[cp_callback])

Epoch 1/100
48/48 [==============================] - 1s 3ms/step - loss: 0.3835 - accuracy: 0.0000e+00
Epoch 2/100
48/48 [==============================] - 0s 4ms/step - loss: 0.0913 - accuracy: 0.0000e+00
Epoch 3/100
48/48 [==============================] - 0s 3ms/step - loss: -0.0182 - accuracy: 0.0000e+00
Epoch 4/100
48/48 [==============================] - 0s 4ms/step - loss: -0.1534 - accuracy: 0.0000e+00
Epoch 5/100
48/48 [==============================] - 0s 4ms/step - loss: -0.3661 - accuracy: 0.0000e+00
Epoch 6/100
48/48 [==============================] - 0s 4ms/step - loss: -0.6847 - accuracy: 0.0000e+00
Epoch 7/100
48/48 [==============================] - 0s 4ms/step - loss: -1.2070 - accuracy: 0.0000e+00
Epoch 8/100
48/48 [==============================] - 0s 4ms/step - loss: -2.1206 - accuracy: 0.0000e+00
Epoch 9/100
48/48 [==============================] - 0s 4ms/step - loss: -3.5462 - accuracy: 0.0000e+00
Epoch 10/100
48/48 [==============================] - 0s 4ms/step 

In [20]:
# Prediction and check performance metrics
train_predict=nn.predict(X_train)
test_predict=nn.predict(X_test)

In [21]:
# Test Data RMSE
math.sqrt(mean_squared_error(y_test,test_predict))

1.3690970596909957

In [22]:
print(metrics.mean_absolute_error (y_test, test_predict))

0.9157356246588929


In [23]:
# shift train predictions for plotting
look_back=100
trainPredictPlot = np.empty_like(stock_df1)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(train_predict)+look_back, :] = train_predict
# shift test predictions for plotting
testPredictPlot = np.empty_like(stock_df1)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(train_predict)+(look_back*2)+1:len(stock_df1)-1, :] = test_predict

In [24]:
Test_Data = pd.DataFrame(data = testPredictPlot, columns = ['Test',]).dropna()
Training_Data = pd.DataFrame(trainPredictPlot, columns = ['Training']).dropna()

In [25]:
predection_df = pd.concat([stock_data, Test_Data, Training_Data], axis = 1)

In [26]:
fig1 = go.Figure()

fig1.add_trace(
    go.Scatter(x=list(predection_df.Date), y=list(predection_df.MMM), name='Stock Returns'))

fig1.add_trace(
    go.Scatter(x=list(predection_df.Date), y=list(predection_df.Test), name='Test'))

fig1.add_trace(
    go.Scatter(x=list(predection_df.Date), y=list(predection_df.Training), name='Training'))


fig1.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1,
                     label="1m",
                     step="month",
                     stepmode="backward"),
                dict(count=6,
                     label="6m",
                     step="month",
                     stepmode="backward"),
                dict(count=1,
                     label="YTD",
                     step="year",
                     stepmode="todate"),
                dict(count=1,
                     label="1y",
                     step="year",
                     stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(
            visible=True
        ),
        type="date"
    )
)

fig1.show()